In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

from selenium.webdriver import ChromeOptions

option = ChromeOptions()

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_argument('--no-sandbox')
option.add_argument(UA.get_User_Agent_Selenium())
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--headless=new')
option.add_extension(Proxy.get_Proxy_Selenium('coroutine'))

option.page_load_strategy = 'eager'

import time

# = = = = = = = = = = = = = = =

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'
        for b in range(10):
            try:
                browser = Chrome(service=Service('../../../chromedriver'), options=option)
                browser.maximize_window()
                browser.get(input_.loc[a, 'Url'])

                time.sleep(1)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                time.sleep(1)
                browser.execute_script('window.scrollTo(0, -document.body.scrollHeight);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.1);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.2);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.3);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.4);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.5);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.6);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.7);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.8);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.9);')
                time.sleep(0.2)
                browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                time.sleep(1)

                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="product-name"]/h1')))
                title = browser.find_element(by=By.XPATH, value='//div[@class="product-name"]/h1').text.strip()

                # = = = = = = = = = = = = = = =

                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="additional-details"]/div')))
                list_div = browser.find_elements(by=By.XPATH, value='//div[@class="additional-details"]/div')

                # = = = = = = = = = = = = = = =

                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="manufacturers"]')))
                list_div_2 = browser.find_elements(by=By.XPATH, value='//div[@class="manufacturers"]')

                # = = = = = = = = = = = = = = =

                price = '' if len(browser.find_elements(by=By.XPATH, value='//div[@class="product-price"]/span')) == 0 else ' '.join([span.text.strip() for span in browser.find_elements(by=By.XPATH, value='//div[@class="product-price"]/span')])

                # = = = = = = = = = = = = = = =

                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="picture"]/img')))
                src = browser.find_element(by=By.XPATH, value='//div[@class="picture"]/img').get_attribute('src').strip()

                # = = = = = = = = = = = = = = =
                
                oe = '' if len(browser.find_elements(by=By.XPATH, value='//div[@id="oeEquivalents"]')) == 0 else browser.find_element(by=By.XPATH, value='//div[@id="oeEquivalents"]').get_attribute('textContent').replace('OE#', '').strip()

                # = = = = = = = = = = = = = = =
                
                list_li = []
                wait.until(EC.presence_of_element_located((By.XPATH, '//div[@id="productSpecifications"]/ul')))
                list_ul = browser.find_elements(by=By.XPATH, value='//div[@id="productSpecifications"]/ul')
                for ul in list_ul:
                    list_li += ul.find_elements(by=By.XPATH, value='./li')[1:]
    
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                list_li_2 = browser.find_elements(by=By.XPATH, value='//ul[@id="product-fitment-accordion"]/li')
                for li_2 in list_li_2:
                    year= int(li_2.find_element(by=By.XPATH, value='./a').get_attribute('textContent').strip())
                    list_li_3 = li_2.find_elements(by=By.XPATH, value='./div/ul/li')
                    for li_3 in list_li_3:
                        make = li_3.find_element(by=By.XPATH, value='./a').get_attribute('textContent').strip()
                        list_model = [li.get_attribute('textContent').strip() for li in li_3.find_elements(by=By.XPATH, value='./div/ul/li/span')]
                        for model in list_model:
                            if make not in dict_vehicle:
                                dict_vehicle[make] = {model: [year]}
                            else:
                                if model not in dict_vehicle[make]:
                                    dict_vehicle[make][model] = [year]
                                else:
                                    dict_vehicle[make][model].append(year)
                                    dict_vehicle[make][model] = sorted(list(set(dict_vehicle[make][model])))

                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        year_min = min(dict_vehicle[make][model])
                        year_max = max(dict_vehicle[make][model])
                        if year_min == year_max:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                        else:
                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                vehicle = '\n'.join(sorted(list_vehicle))

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Title': title,
                                         'Price': price,
                                         'Vehicle': vehicle,
                                         'OE': oe,
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src}])

                for div in list_div:
                    df_temp.loc[0, div.find_elements(by=By.XPATH, value='./span')[0].text.strip()] = div.find_elements(by=By.XPATH, value='./span')[1].text.strip()

                for div in list_div_2:
                    df_temp.loc[0, div.find_elements(by=By.XPATH, value='./span')[0].text.strip()] = div.find_elements(by=By.XPATH, value='./span')[1].text.strip()

                for li in list_li:
                    df_temp.loc[0, li.find_elements(by=By.XPATH, value='./span')[0].text.strip()] = li.find_elements(by=By.XPATH, value='./span')[1].text.strip()

                browser.quit()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                break
            except:
                browser.quit()
                time.sleep(0.3)
                continue

                # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'序号': int(input_.loc[a, '序号']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：642

[ok] - https://www.densoproducts.com/denso-999-0507-ac-o-ring
[尝试次数：1] - [剩余数量：635] - [当前时间：17:31:25]

[ok] - https://www.densoproducts.com/denso-478-2107-receiver-drier
[尝试次数：1] - [剩余数量：634] - [当前时间：17:31:31]

[ok] - https://www.densoproducts.com/denso-999-0115-ac-plate
[尝试次数：1] - [剩余数量：633] - [当前时间：17:31:31]

[ok] - https://www.densoproducts.com/denso-999-0116-ac-suction-plug
[尝试次数：1] - [剩余数量：632] - [当前时间：17:31:32]

[ok] - https://www.densoproducts.com/denso-999-0506-ac-o-ring
[尝试次数：1] - [剩余数量：631] - [当前时间：17:31:33]

[ok] - https://www.densoproducts.com/denso-478-8111-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：630] - [当前时间：17:31:33]

[ok] - https://www.densoproducts.com/denso-478-9519-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：629] - [当前时间：17:31:34]

[ok] - https://www.densoproducts.com/denso-478-9518-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：628] - [当前时间：17:31:38]

[ok] - https://www.densoproducts.com/denso-999-0113-ac-plate
[尝试次数：1] - [剩余数量：627] - [当前时间：17:31:47]

[ok]

[ok] - https://www.densoproducts.com/denso-478-2091-receiver-drier
[尝试次数：2] - [剩余数量：562] - [当前时间：17:34:11]

[ok] - https://www.densoproducts.com/denso-478-9505-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：561] - [当前时间：17:34:11]

[ok] - https://www.densoproducts.com/denso-478-9544-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：560] - [当前时间：17:34:13]

[ok] - https://www.densoproducts.com/denso-478-6504-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：559] - [当前时间：17:34:18]

[ok] - https://www.densoproducts.com/denso-478-9543-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：558] - [当前时间：17:34:26]

[ok] - https://www.densoproducts.com/denso-999-0305-ac-discharge-o-ring
[尝试次数：1] - [剩余数量：557] - [当前时间：17:34:26]

[ok] - https://www.densoproducts.com/denso-478-2090-receiver-drier
[尝试次数：2] - [剩余数量：556] - [当前时间：17:34:27]

[ok] - https://www.densoproducts.com/denso-478-9503-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：555] - [当前时间：17:34:27]

[ok] - https://www.densoproducts.com/denso-478-6503-receiver-drier-accu

[ok] - https://www.densoproducts.com/denso-478-8509-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：490] - [当前时间：17:37:14]

[ok] - https://www.densoproducts.com/denso-478-3054-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：489] - [当前时间：17:37:17]

[ok] - https://www.densoproducts.com/denso-478-2076-receiver-drier
[尝试次数：1] - [剩余数量：488] - [当前时间：17:37:18]

[ok] - https://www.densoproducts.com/denso-478-9529-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：487] - [当前时间：17:37:25]

[ok] - https://www.densoproducts.com/denso-999-0205-ac-o-ring
[尝试次数：1] - [剩余数量：486] - [当前时间：17:37:26]

[ok] - https://www.densoproducts.com/denso-478-8508-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：485] - [当前时间：17:37:28]

[ok] - https://www.densoproducts.com/denso-478-9538-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：484] - [当前时间：17:37:30]

[ok] - https://www.densoproducts.com/denso-478-2075-receiver-drier
[尝试次数：1] - [剩余数量：483] - [当前时间：17:37:32]

[ok] - https://www.densoproducts.com/denso-478-3028-accumulator
[尝试次数：1] - [剩余数量：

[ok] - https://www.densoproducts.com/denso-478-2046-receiver-drier
[尝试次数：2] - [剩余数量：416] - [当前时间：17:40:22]

[ok] - https://www.densoproducts.com/denso-478-2043-receiver-drier
[尝试次数：3] - [剩余数量：415] - [当前时间：17:40:26]

[ok] - https://www.densoproducts.com/denso-478-2041-receiver-drier
[尝试次数：1] - [剩余数量：414] - [当前时间：17:40:27]

[ok] - https://www.densoproducts.com/denso-478-2042-receiver-drier
[尝试次数：1] - [剩余数量：413] - [当前时间：17:40:30]

[ok] - https://www.densoproducts.com/denso-478-2039-receiver-drier
[尝试次数：1] - [剩余数量：412] - [当前时间：17:40:33]

[ok] - https://www.densoproducts.com/denso-478-2040-receiver-drier
[尝试次数：2] - [剩余数量：411] - [当前时间：17:40:35]

[ok] - https://www.densoproducts.com/denso-478-2037-receiver-drier
[尝试次数：1] - [剩余数量：410] - [当前时间：17:40:37]

[ok] - https://www.densoproducts.com/denso-478-2035-receiver-drier
[尝试次数：1] - [剩余数量：409] - [当前时间：17:40:39]

[ok] - https://www.densoproducts.com/denso-478-2036-receiver-drier
[尝试次数：1] - [剩余数量：408] - [当前时间：17:40:41]

[ok] - https://www.densoprod

[ok] - https://www.densoproducts.com/denso-476-0084-evaporator-core
[尝试次数：1] - [剩余数量：340] - [当前时间：17:43:43]

[ok] - https://www.densoproducts.com/denso-476-0085-evaporator-core
[尝试次数：1] - [剩余数量：339] - [当前时间：17:43:45]

[ok] - https://www.densoproducts.com/denso-476-0082-evaporator-core
[尝试次数：1] - [剩余数量：338] - [当前时间：17:43:45]

[ok] - https://www.densoproducts.com/denso-476-0080-evaporator-core
[尝试次数：1] - [剩余数量：337] - [当前时间：17:43:45]

[ok] - https://www.densoproducts.com/denso-476-0081-evaporator-core
[尝试次数：1] - [剩余数量：336] - [当前时间：17:43:53]

[ok] - https://www.densoproducts.com/denso-476-0077-evaporator-core
[尝试次数：1] - [剩余数量：335] - [当前时间：17:43:56]

[ok] - https://www.densoproducts.com/denso-476-0078-evaporator-core
[尝试次数：2] - [剩余数量：334] - [当前时间：17:44:01]

[ok] - https://www.densoproducts.com/denso-476-0075-evaporator-core
[尝试次数：1] - [剩余数量：333] - [当前时间：17:44:01]

[ok] - https://www.densoproducts.com/denso-476-0073-evaporator-core
[尝试次数：1] - [剩余数量：332] - [当前时间：17:44:07]

[ok] - https://www.

[ok] - https://www.densoproducts.com/denso-476-0003-evaporator-core
[尝试次数：1] - [剩余数量：264] - [当前时间：17:47:20]

[ok] - https://www.densoproducts.com/denso-476-0004-evaporator-core
[尝试次数：2] - [剩余数量：263] - [当前时间：17:47:20]

[ok] - https://www.densoproducts.com/denso-476-0012-evaporator-core
[尝试次数：1] - [剩余数量：262] - [当前时间：17:47:21]

[ok] - https://www.densoproducts.com/denso-476-0002-evaporator-core
[尝试次数：1] - [剩余数量：261] - [当前时间：17:47:22]

[ok] - https://www.densoproducts.com/denso-475-7513-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：260] - [当前时间：17:47:29]

[ok] - https://www.densoproducts.com/denso-476-0001-evaporator-core
[尝试次数：2] - [剩余数量：259] - [当前时间：17:47:29]

[ok] - https://www.densoproducts.com/denso-476-0010-evaporator-core
[尝试次数：1] - [剩余数量：258] - [当前时间：17:47:31]

[ok] - https://www.densoproducts.com/denso-475-7506-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：257] - [当前时间：17:47:34]

[ok] - https://www.densoproducts.com/denso-475-6508-receiver-drier-accumulator
[尝试次数：1] - [剩余数量：256] - [当前

[ok] - https://www.densoproducts.com/denso-473-0331-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：192] - [当前时间：17:50:24]

[ok] - https://www.densoproducts.com/denso-473-0330-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：191] - [当前时间：17:50:25]

[ok] - https://www.densoproducts.com/denso-473-0329-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：190] - [当前时间：17:50:30]

[ok] - https://www.densoproducts.com/denso-473-0333-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：189] - [当前时间：17:50:31]

[ok] - https://www.densoproducts.com/denso-473-0326-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：188] - [当前时间：17:50:33]

[ok] - https://www.densoproducts.com/denso-473-0328-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：187] - [当前时间：17:50:34]

[ok] - https://www.densoproducts.com/denso-473-0324-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：186] - [当前时间：17:50:36]

[ok] - https://www.densoproducts.com/denso-473-0325-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：185] - [当前时间：17:50:39]

[ok] - https://www.densoproducts.com/denso-473-0322-rema

[ok] - https://www.densoproducts.com/denso-473-0244-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：122] - [当前时间：17:53:12]

[ok] - https://www.densoproducts.com/denso-473-0241-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：121] - [当前时间：17:53:19]

[ok] - https://www.densoproducts.com/denso-473-0240-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：120] - [当前时间：17:53:22]

[ok] - https://www.densoproducts.com/denso-473-0238-remanufactured-ac-clutch
[尝试次数：2] - [剩余数量：119] - [当前时间：17:53:27]

[ok] - https://www.densoproducts.com/denso-473-0239-remanufactured-ac-clutch
[尝试次数：2] - [剩余数量：118] - [当前时间：17:53:34]

[ok] - https://www.densoproducts.com/denso-473-0237-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：117] - [当前时间：17:53:34]

[ok] - https://www.densoproducts.com/denso-473-0236-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：116] - [当前时间：17:53:37]

[ok] - https://www.densoproducts.com/denso-473-0233-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：115] - [当前时间：17:53:46]

[ok] - https://www.densoproducts.com/denso-473-0235-rema

[ok] - https://www.densoproducts.com/denso-473-0161-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：52] - [当前时间：17:56:26]

[ok] - https://www.densoproducts.com/denso-473-0160-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：51] - [当前时间：17:56:31]

[ok] - https://www.densoproducts.com/denso-473-0159-remanufactured-ac-clutch
[尝试次数：2] - [剩余数量：50] - [当前时间：17:56:48]

[ok] - https://www.densoproducts.com/denso-473-0157-remanufactured-ac-clutch
[尝试次数：2] - [剩余数量：49] - [当前时间：17:56:51]

[ok] - https://www.densoproducts.com/denso-473-0155-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：48] - [当前时间：17:56:54]

[ok] - https://www.densoproducts.com/denso-473-0158-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：47] - [当前时间：17:56:55]

[ok] - https://www.densoproducts.com/denso-473-0154-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：46] - [当前时间：17:56:55]

[ok] - https://www.densoproducts.com/denso-473-0156-remanufactured-ac-clutch
[尝试次数：1] - [剩余数量：45] - [当前时间：17:56:56]

[ok] - https://www.densoproducts.com/denso-473-0152-remanufactur